In [0]:
dbutils.widgets.dropdown(name = 'environment', choices= ['dev', 'uat', 'prod'], defaultValue = 'dev', label = 'select your environment')
environment = dbutils.widgets.get('environment')

In [0]:
# %pip install great_expectations==1.10.0

In [0]:
#dbutils.library.restartPython()

In [0]:
%run "./04_common"

In [0]:
import great_expectations_common as gec
print(dir(gec))

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
schema_traffic = StructType([
        StructField("Record_ID",IntegerType()),
        StructField("Count_point_id",IntegerType()),
        StructField("Direction_of_travel",StringType()),
        StructField("Year",IntegerType()),
        StructField("Count_date",StringType()),
        StructField("hour",IntegerType()),
        StructField("Region_id",IntegerType()),
        StructField("Region_name",StringType()),
        StructField("Local_authority_name",StringType()),
        StructField("Road_name",StringType()),
        StructField("Road_Category_ID",IntegerType()),
        StructField("Start_junction_road_name",StringType()),
        StructField("End_junction_road_name",StringType()),
        StructField("Latitude",DoubleType()),
        StructField("Longitude",DoubleType()),
        StructField("Link_length_km",DoubleType()),
        StructField("Pedal_cycles",IntegerType()),
        StructField("Two_wheeled_motor_vehicles",IntegerType()),
        StructField("Cars_and_taxis",IntegerType()),
        StructField("Buses_and_coaches",IntegerType()),
        StructField("LGV_Type",IntegerType()),
        StructField("HGV_Type",IntegerType()),
        StructField("EV_Car",IntegerType()),
        StructField("EV_Bike",IntegerType())
    ])

In [0]:
schema_roads =  StructType([
        StructField('Road_ID',IntegerType()),
        StructField('Road_Category_Id',IntegerType()),
        StructField('Road_Category',StringType()),
        StructField('Region_ID',IntegerType()),
        StructField('Region_Name',StringType()),
        StructField('Total_Link_Length_Km',DoubleType()),
        StructField('Total_Link_Length_Miles',DoubleType()),
        StructField('All_Motor_Vehicles',DoubleType())
        
        ])

In [0]:
def load_raw_traffic(schema):
    rawTraffic_stream = spark.readStream\
                    .format('cloudFiles')\
                    .option('cloudFiles.format', 'csv')\
                    .option('cloudFiles.schemaLocation', checkpoint_url + 'rawTraffic_load/schema')\
                    .option('header', 'true')\
                    .option("pathGlobFilter", "raw_traffic*.csv")\
                    .schema(schema)\
                    .load(landing_url)\
                    .withColumn('Extract_Time', current_timestamp())
    print('read raw traffic successfully')
    return rawTraffic_stream

In [0]:
def load_raw_roads(schema):
    rawTraffic_stream = spark.readStream\
                    .format('cloudFiles')\
                    .option('cloudFiles.format', 'csv')\
                    .option('cloudFiles.schemaLocation', checkpoint_url + 'rawRoads_load/schema')\
                    .option('header', 'true')\
                    .option("pathGlobFilter", "raw_roads*.csv")\
                    .schema(schema)\
                    .load(landing_url)
    print('read raw roads successfully')
    return rawTraffic_stream

In [0]:
def write_raw_traffic(rawTraffic_stream, environment):
    catalog_name = f"traffic_{environment}_catalog"
    schema_name = db_name 
    table_name = "raw_traffic"
    
    rawTraffic_stream.writeStream \
        .foreachBatch(lambda batch_df, b_id: gec.validate_and_insert_process_batch(
            df=batch_df,           
            batch_id=b_id,         
            table_name=table_name, 
            catalog=catalog_name,  
            schema=schema_name     
        )) \
        .option("checkpointLocation", checkpoint_url + 'rawTraffic_load/checkpoint') \
        .trigger(availableNow=True) \
        .start()
    
    print(f'Successfully started writing {table_name} to {catalog_name}')


In [0]:
def write_raw_roads(rawRoads_stream, environment):
    catalog_name = f"traffic_{environment}_catalog"
    schema_name = db_name 
    table_name = "raw_roads"
    
    rawRoads_stream.writeStream \
        .foreachBatch(lambda batch_df, b_id: gec.validate_and_insert_process_batch(
            df=batch_df,           
            batch_id=b_id,         
            table_name=table_name, 
            catalog=catalog_name,  
            schema=schema_name     
        )) \
        .option("checkpointLocation", checkpoint_url + 'rawRoads_load/checkpoint') \
        .trigger(availableNow=True) \
        .start()
    
    print(f'Successfully started writing {table_name} to {catalog_name}')

In [0]:
rawTraffic_stream = load_raw_traffic(schema_traffic)
rawRoads_stream = load_raw_roads(schema_roads)



In [0]:
write_raw_traffic(rawTraffic_stream, environment)

In [0]:
write_raw_roads(rawRoads_stream,environment)

In [0]:
# target_schema = StructType([
#         StructField("Record_ID",IntegerType()),
#         StructField("Count_point_id",IntegerType()),
#         StructField("Direction_of_travel",StringType()),
#         StructField("Year",IntegerType()),
#         StructField("Count_date",StringType()),
#         StructField("hour",IntegerType()),
#         StructField("Region_id",IntegerType()),
#         StructField("Region_name",StringType()),
#         StructField("Local_authority_name",StringType()),
#         StructField("Road_name",StringType()),
#         StructField("Road_Category_ID",IntegerType()),
#         StructField("Start_junction_road_name",StringType()),
#         StructField("End_junction_road_name",StringType()),
#         StructField("Latitude",DoubleType()),
#         StructField("Longitude",DoubleType()),
#         StructField("Link_length_km",DoubleType()),
#         StructField("Pedal_cycles",IntegerType()),
#         StructField("Two_wheeled_motor_vehicles",IntegerType()),
#         StructField("Cars_and_taxis",IntegerType()),
#         StructField("Buses_and_coaches",IntegerType()),
#         StructField("LGV_Type",IntegerType()),
#         StructField("HGV_Type",IntegerType()),
#         StructField("EV_Car",IntegerType()),
#         StructField("EV_Bike",IntegerType()),
#         StructField("Extract_Time",TimestampType())
#     ])

In [0]:
# from pyspark.sql import functions as F

# quarantine_df = spark.table("traffic_dev_catalog.gx.data_quality_quarantine") \
#         .filter(F.col("table_name") == "raw_traffic") \
#         .filter(F.col("violated_rules") == "[Count_date] expect_column_values_to_match_strftime_format") 

# if quarantine_df.count() == 0:
#     print(f"没有发现表 raw_traffic 的隔离数据。")


# recovered_df = quarantine_df.withColumn("parsed_data", F.from_json(F.col("raw_data"), target_schema)) \
#         .select("parsed_data.*")
# display(recovered_df)

In [0]:
# def write_raw_traffic_recovery(df, environment):
#     catalog_name = f"traffic_{environment}_catalog"
#     schema_name = db_name 
#     table_name = "raw_traffic"
    
#     # 直接手动执行验证和插入逻辑，给 batch_id 一个固定值即可
#     gec.validate_and_insert_process_batch(
#         df=df,           
#         batch_id=1,         
#         table_name=table_name, 
#         catalog=catalog_name,  
#         schema=schema_name     
#     )
    
#     print(f'Successfully processed one-time batch for {table_name} to {catalog_name}')

In [0]:
write_raw_traffic_recovery(recovered_df, environment)

In [0]:
# from delta.tables import DeltaTable

# # 加载表对象
# deltaTable = DeltaTable.forName(spark, "traffic_dev_catalog.gx.data_quality_quarantine")

# # 执行删除
# deltaTable.delete(
#     condition = "table_name = 'raw_traffic' AND violated_rules = '[Count_date] expect_column_values_to_match_strftime_format'"
# )